<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class Controller(nn.Module):
    def __init__(self, search_space):
        super(Controller, self).__init__()
        self.search_space = search_space
        self.lstm = nn.LSTM(input_size=len(search_space), hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, len(search_space))

    def forward(self, x, hidden=None):
        x, hidden = self.lstm(x, hidden)
        x = self.fc(x)
        return x, hidden

def sample_architecture(controller, search_space, device):
    architecture = []
    x = torch.zeros((1, 1, len(search_space)), device=device)  # Start with a zero input
    hidden = None
    for step in range(len(search_space)):
        logits, hidden = controller(x, hidden)  # Forward pass through LSTM
        prob = nn.Softmax(dim=-1)(logits.squeeze(0))
        choice = torch.multinomial(prob, 1).item()  # Sample an action based on probabilities
        architecture.append(search_space[choice])

        # Prepare input for the next step
        x = torch.zeros((1, 1, len(search_space)), device=device)
        x[0, 0, choice] = 1
        print(f"Step {step + 1}, Probabilities: {prob.detach().cpu().numpy()}, Choice: {search_space[choice]}")

    return architecture

# Define search space
search_space = ['conv3x3', 'conv5x5', 'maxpool3x3']

# Initialize Controller and move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
controller = Controller(search_space).to(device)

# Example architecture sampling
architecture = sample_architecture(controller, search_space, device)
print("Sampled Architecture:", architecture)